In [1]:
import os,json
import torch
import torch.nn as nn
from changechat.model.multimodal_encoder.clip_encoder import CLIPVisionTower
from dataclasses import dataclass
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.optim as optim
from torch.nn import CrossEntropyLoss

/root/miniconda3/envs/geochat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-08-07 22:17:39,155] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


## 定义模型

In [2]:
import math
class Classifier(nn.Module):
    def __init__(
        self, mm_hidden_size, hidden_size=None, img_feature_w=18, img_feature_h=18
    ):
        super(Classifier, self).__init__()
        self.d_model = mm_hidden_size

        encoder_self_layer_classifier = nn.TransformerEncoderLayer(
            2 * self.d_model, nhead=2, dim_feedforward=int(2 * self.d_model)
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_self_layer_classifier, num_layers=1
        )
        self.w_embedding = nn.Embedding(img_feature_w, int(self.d_model / 2))
        self.h_embedding = nn.Embedding(img_feature_h, int(self.d_model / 2))
        self.classifier_head = nn.Linear(2 * self.d_model, 2)

        # cls_token
        scale = self.d_model**-0.5
        self.cls_changeflag = nn.Parameter(scale * torch.randn(1, 2 * self.d_model))

    def position_embedding_2D_func(self, img_feat_A, img_feat_B):
        device = img_feat_A.device
        batch = img_feat_B.shape[0]
        Len_feat = img_feat_B.shape[1]
        h = int(math.sqrt(Len_feat))
        w = h
        pos_w = torch.arange(w, device=device).to(device)
        pos_h = torch.arange(h, device=device).to(device)
        embed_w = self.w_embedding(pos_w)
        embed_h = self.h_embedding(pos_h)
        position_embedding = torch.cat(
            [
                embed_w.unsqueeze(0).repeat(h, 1, 1),
                embed_h.unsqueeze(1).repeat(1, w, 1),
            ],
            dim=-1,
        )
        position_embedding = position_embedding.unsqueeze(0).repeat(
            batch, 1, 1, 1
        )  # (batch, h, w, d_model)
        position_embedding = position_embedding.view(batch, -1, self.d_model)
        img_feat_A = img_feat_A + position_embedding  # NLD
        img_feat_B = img_feat_B + position_embedding  # NLD

        return img_feat_A, img_feat_B

    def forward(self, img_feat):
        img_feat_A = img_feat[:, 0, ...]  # (N,L,768)
        img_feat_B = img_feat[:, 1, ...]  # (N,L,768)

        # 2D image position_embedding
        img_feat_A, img_feat_B = self.position_embedding_2D_func(
            img_feat_A, img_feat_B
        )  # (N, L, D)

        img_feat = torch.cat([img_feat_A, img_feat_B], dim=-1)  # (N, L, 2D)
        img_feat_with_cls = torch.cat(
            [
                self.cls_changeflag.unsqueeze(0).expand(
                    img_feat.shape[0], *self.cls_changeflag.shape
                ),
                img_feat,
            ],
            dim=1,
        )

        img_feat_with_cls = self.transformer_encoder(
            img_feat_with_cls.permute(1, 0, 2)
        ).permute(
            1, 0, 2
        )  # (N, L, 2D)
        change_pred = self.classifier_head(img_feat_with_cls[:, 0, :])
        return change_pred

In [3]:
@dataclass
class ChangeClassifierConfig:
    mm_vision_tower = "/root/autodl-tmp/GeoChat/hf-models/clip-vit-large-patch14-336"
    mm_vision_select_layer = -2

class ChangeClassifier(nn.Module):
    def __init__(self, config):
        super(ChangeClassifier, self).__init__()
        self.vision_tower = CLIPVisionTower(config.mm_vision_tower, config)
        # freeeze vision_tower
        self.vision_tower.requires_grad_(False)

        if config.mm_vision_tower.endswith("clip-vit-large-patch14-336"):
            mm_hidden_size = 1024
        else:
            raise NotImplementedError
        self.classifier = Classifier(mm_hidden_size = mm_hidden_size)

    def forward(self, images):
        # 提取两幅图像的特征
        assert images.ndim == 5
        concat_images = torch.cat([image for image in images], dim=0)
        image_features = self.vision_tower(concat_images)
        split_sizes = [image.shape[0] for image in images]
        image_features = torch.split(
            image_features, split_sizes, dim=0
        )  # b tuples of [2, N, L]
        image_features = torch.stack(image_features, dim=0)  # [b, 2, N, L]
        change_pred = self.classifier(image_features)
        return change_pred

## 定义数据集

In [9]:
@dataclass
class DataConfig:
    data_path = "/root/autodl-tmp/LEVIR-MCI-dataset/ChangeChat_classify.json"
    image_folder = "/root/autodl-tmp/LEVIR-MCI-dataset/images"
    image_processor = None

class LazySupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(
        self,
        config
    ):
        super(LazySupervisedDataset, self).__init__()
        list_data_dict = json.load(open(config.data_path, "r"))
        self.list_data_dict = list_data_dict
        self.processor = config.image_processor
        self.image_folder = config.image_folder

    def __len__(self):
        return len(self.list_data_dict)

    def __getitem__(self, i):
        sources = self.list_data_dict[i]
        if isinstance(i, int):
            sources = [sources]
        assert len(sources) == 1, "Don't know why it is wrapped to a list"  # FIXME
        image_file = self.list_data_dict[i]["image"]
        image_folder = self.image_folder
        processor = self.processor

        if isinstance(image_file, str):
            image_file_list = [image_file]
        elif isinstance(image_file, list):
            image_file_list = image_file

        imageList = []
        for _image_file in image_file_list:
            image = Image.open(
                (os.path.join(image_folder, _image_file)).strip()
            ).convert("RGB")
            image = processor.preprocess(
                image,
                do_resize=True,
                crop_size={"height": 252, "width": 252},
                size={"shortest_edge": 252},
                return_tensors="pt",
            )["pixel_values"][0]
            imageList.append(image)

        data_dict = dict()
        # 将多幅图像拼成一个Tensor
        data_dict["images"] = torch.stack(imageList, dim=0)  # (2, c, h, w)

        # 如果有变化标签
        if "changeflag" in self.list_data_dict[i]:
            data_dict["change_labels"] = torch.tensor(
                self.list_data_dict[i]["changeflag2"]
            )
        return data_dict

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = ChangeClassifierConfig()
model = ChangeClassifier(config)
model = model.to(device)
# 创建数据集和数据加载器
dataset_config = DataConfig()
dataset_config.image_processor = model.vision_tower.image_processor
train_dataset = LazySupervisedDataset(dataset_config)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 验证集
dataset_config.data_path = "/root/autodl-tmp/GeoChat/load/Test_CC_gt.json"
test_dataset = LazySupervisedDataset(dataset_config)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

params = [v for v in model.parameters() if v.requires_grad]
num_params = sum([v.numel() for v in model.parameters() if v.requires_grad])/1000000
optimizer = optim.Adam(params, lr=1e-4)
criterion = CrossEntropyLoss()

In [6]:
import wandb
wandb.init(project="change_classifier", config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hanlinwu. Use `wandb login --relogin` to force relogin


In [7]:
import tqdm
def calculate_accuracy(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm.tqdm(data_loader):
            images, labels = batch['images'].to(device), batch['change_labels'].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

In [11]:
val_accuracy = calculate_accuracy(model, test_loader, device)
print(f"Validation Accuracy: {val_accuracy:.4f}")

  0%|          | 0/16 [00:00<?, ?it/s]


KeyError: 'changeflag2'

In [8]:
import datetime
num_epochs = 400
print_step = 10
global_step = 0
output_dir = "/root/autodl-tmp/GeoChat/experiments/classifier"
now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = output_dir + now
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, batch in enumerate(train_loader):
        images, labels = batch['images'].to(device), batch['change_labels'].to(device)
        
        # 零梯度
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(images)
        
        # 计算损失
        loss = criterion(outputs, labels)
        
        # 反向传播
        loss.backward()
        
        # 更新参数
        optimizer.step()
        global_step += 1
        
        running_loss += loss.item()

        if (i+1)%print_step == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")
            wandb.log({
                "epoch": epoch + 1,
                "global_step": global_step,
                "loss": loss.item()
            })
    
    # 打印每个epoch的平均损失
    val_accuracy = calculate_accuracy(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}] Validation Accuracy: {val_accuracy:.4f}")
    torch.save(model.classifier.state_dict(), os.path.join(output_dir,"classifier_weight.pth"))
    wandb.log({
        "epoch": epoch + 1,
        "val_accuracy": val_accuracy,
    })

wandb.finish()

Epoch [1/400], Step [10/213], Loss: 0.6896
Epoch [1/400], Step [20/213], Loss: 0.8022
Epoch [1/400], Step [30/213], Loss: 0.5419
Epoch [1/400], Step [40/213], Loss: 0.6253
Epoch [1/400], Step [50/213], Loss: 0.4691
Epoch [1/400], Step [60/213], Loss: 0.3945
Epoch [1/400], Step [70/213], Loss: 0.3566
Epoch [1/400], Step [80/213], Loss: 0.3365
Epoch [1/400], Step [90/213], Loss: 0.2182
Epoch [1/400], Step [100/213], Loss: 0.1896
Epoch [1/400], Step [110/213], Loss: 0.3334
Epoch [1/400], Step [120/213], Loss: 0.3010
Epoch [1/400], Step [130/213], Loss: 0.2211
Epoch [1/400], Step [140/213], Loss: 0.1551
Epoch [1/400], Step [150/213], Loss: 0.5036
Epoch [1/400], Step [160/213], Loss: 0.1395
Epoch [1/400], Step [170/213], Loss: 0.3048
Epoch [1/400], Step [180/213], Loss: 0.2300
Epoch [1/400], Step [190/213], Loss: 0.2855
Epoch [1/400], Step [200/213], Loss: 0.2224
Epoch [1/400], Step [210/213], Loss: 0.2734


100%|██████████| 16/16 [01:13<00:00,  4.59s/it]


Epoch [1/400] Validation Accuracy: 0.9160
Epoch [2/400], Step [10/213], Loss: 0.1945
Epoch [2/400], Step [20/213], Loss: 0.0991
Epoch [2/400], Step [30/213], Loss: 0.1677
Epoch [2/400], Step [40/213], Loss: 0.2006
Epoch [2/400], Step [50/213], Loss: 0.2458
Epoch [2/400], Step [60/213], Loss: 0.2479
Epoch [2/400], Step [70/213], Loss: 0.1665
Epoch [2/400], Step [80/213], Loss: 0.2897
Epoch [2/400], Step [90/213], Loss: 0.2025
Epoch [2/400], Step [100/213], Loss: 0.3231
Epoch [2/400], Step [110/213], Loss: 0.2127
Epoch [2/400], Step [120/213], Loss: 0.1730
Epoch [2/400], Step [130/213], Loss: 0.2556
Epoch [2/400], Step [140/213], Loss: 0.4395
Epoch [2/400], Step [150/213], Loss: 0.3377
Epoch [2/400], Step [160/213], Loss: 0.1409
Epoch [2/400], Step [170/213], Loss: 0.1532
Epoch [2/400], Step [180/213], Loss: 0.2074
Epoch [2/400], Step [190/213], Loss: 0.0988
Epoch [2/400], Step [200/213], Loss: 0.1294
Epoch [2/400], Step [210/213], Loss: 0.0997


100%|██████████| 16/16 [01:13<00:00,  4.58s/it]


Epoch [2/400] Validation Accuracy: 0.8979
Epoch [3/400], Step [10/213], Loss: 0.1188
Epoch [3/400], Step [20/213], Loss: 0.1742
Epoch [3/400], Step [30/213], Loss: 0.1341
Epoch [3/400], Step [40/213], Loss: 0.2538
Epoch [3/400], Step [50/213], Loss: 0.1759
Epoch [3/400], Step [60/213], Loss: 0.3366
Epoch [3/400], Step [70/213], Loss: 0.2801
Epoch [3/400], Step [80/213], Loss: 0.1236
Epoch [3/400], Step [90/213], Loss: 0.2401
Epoch [3/400], Step [100/213], Loss: 0.1453
Epoch [3/400], Step [110/213], Loss: 0.0829
Epoch [3/400], Step [120/213], Loss: 0.2364
Epoch [3/400], Step [130/213], Loss: 0.1724
Epoch [3/400], Step [140/213], Loss: 0.0853
Epoch [3/400], Step [150/213], Loss: 0.1240
Epoch [3/400], Step [160/213], Loss: 0.1341
Epoch [3/400], Step [170/213], Loss: 0.2044
Epoch [3/400], Step [180/213], Loss: 0.2015
Epoch [3/400], Step [190/213], Loss: 0.0971
Epoch [3/400], Step [200/213], Loss: 0.3246
Epoch [3/400], Step [210/213], Loss: 0.3627


...

100%|██████████| 16/16 [01:13<00:00,  4.60s/it]


Epoch [17/400] Validation Accuracy: 0.9243
Epoch [18/400], Step [10/213], Loss: 0.0475
Epoch [18/400], Step [20/213], Loss: 0.0095
Epoch [18/400], Step [30/213], Loss: 0.1552
Epoch [18/400], Step [40/213], Loss: 0.1455
Epoch [18/400], Step [50/213], Loss: 0.0353
Epoch [18/400], Step [60/213], Loss: 0.0270
Epoch [18/400], Step [70/213], Loss: 0.0652
Epoch [18/400], Step [80/213], Loss: 0.1042
Epoch [18/400], Step [90/213], Loss: 0.0671
Epoch [18/400], Step [100/213], Loss: 0.0010
Epoch [18/400], Step [110/213], Loss: 0.1244
Epoch [18/400], Step [120/213], Loss: 0.0283
Epoch [18/400], Step [130/213], Loss: 0.1755
Epoch [18/400], Step [140/213], Loss: 0.0156
Epoch [18/400], Step [150/213], Loss: 0.0607
Epoch [18/400], Step [160/213], Loss: 0.1087
Epoch [18/400], Step [170/213], Loss: 0.1682
Epoch [18/400], Step [180/213], Loss: 0.0951
Epoch [18/400], Step [190/213], Loss: 0.1952
Epoch [18/400], Step [200/213], Loss: 0.1702
Epoch [18/400], Step [210/213], Loss: 0.0882


100%|██████████| 16/16 [01:13<00:00,  4.58s/it]


Epoch [18/400] Validation Accuracy: 0.9228
Epoch [19/400], Step [10/213], Loss: 0.1810
Epoch [19/400], Step [20/213], Loss: 0.0540
Epoch [19/400], Step [30/213], Loss: 0.0107
Epoch [19/400], Step [40/213], Loss: 0.0455
Epoch [19/400], Step [50/213], Loss: 0.1461
Epoch [19/400], Step [60/213], Loss: 0.1280
Epoch [19/400], Step [70/213], Loss: 0.0813
Epoch [19/400], Step [80/213], Loss: 0.2084
Epoch [19/400], Step [90/213], Loss: 0.0589
Epoch [19/400], Step [100/213], Loss: 0.0477
Epoch [19/400], Step [110/213], Loss: 0.0437
Epoch [19/400], Step [120/213], Loss: 0.0921
Epoch [19/400], Step [130/213], Loss: 0.1261
Epoch [19/400], Step [140/213], Loss: 0.0376


KeyboardInterrupt: 